https://github.com/earth-chris/earthlib/blob/main/notebooks/CompareAusKenyaData.ipynb

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#pip install geemap
#conda install -c conda-forge earthengine-api
import ee
import geemap

In [ ]:
ee.Authenticate()

In [ ]:
# initialize earth engine api access (required on every run)
ee.Initialize()

# test whether the initialization was successful by printing the elevation of Mount Everest
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Point location elevation (m):', elev)

In [ ]:
js_snippet = """
var Tree_cover = ee.ImageCollection("NASA/MEASURES/GFCC/TC/v3");
var SA = ee.FeatureCollection("users/liu02034/SanAntonioTX_Project");
var NLCD = ee.ImageCollection("USGS/NLCD_RELEASES/2019_REL/NLCD");

var batch = require('users/fitoprincipe/geetools:batch');
var region = SA;

/*Map.addLayer(Tanu, {}, 'Tanu', false); 
Map.addLayer(Joe, {}, 'Joe', false); */

Map.addLayer(SA, {}, 'SA', false);

var GDriveOutputImgFolder = 'GEEOutputs'; 

//tree canopy
var FirstYear = 2015;
var LastYear = 2015;
var SelectedVariableName = 'tree_canopy_cover'; 
var coll1 =  Tree_cover.filterDate(String(FirstYear)+'-01-01', String(LastYear)+'-12-31').filterBounds(region).select([SelectedVariableName]); 
print("ori",coll1);   
//Map.addLayer(coll1, {}, 'tree_cover', false); 

var mosaic = coll1.mosaic();
Map.addLayer(mosaic, {}, 'mosaic', false); 
print("mosaic",mosaic); 

//NLCD 2016
var FirstYear = 2016;
var LastYear = 2016;
var SelectedVariableName = 'landcover'; 
var coll1_NLCD =  NLCD.filterDate(String(FirstYear)+'-01-01', String(LastYear)+'-12-31').filterBounds(region).select([SelectedVariableName]); 
print("ori_NLCD",coll1_NLCD);   

var mosaic_NLCD = coll1_NLCD.mosaic();
Map.addLayer(mosaic_NLCD, {}, 'mosaic_NLCD', false); 
print("mosaic_NLCD",mosaic_NLCD); 


Export.image.toDrive({
      image: mosaic,
      description: 'Tree_canopy_NASA_30m_SA',
      scale: 30,
      region: region,
      maxPixels: 1.0e13,
      folder: GDriveOutputImgFolder,
    });
    
Export.image.toDrive({
      image: mosaic_NLCD,
      description: 'NLCD_2016_30m_SA',
      scale: 30,
      region: region,
      maxPixels: 1.0e13,
      folder: GDriveOutputImgFolder,
    });

"""

In [ ]:
geemap.js_snippet_to_py(js_snippet)

In [ ]:
import ee
import geemap

Map = geemap.Map()

Tree_cover = ee.ImageCollection("NASA/MEASURES/GFCC/TC/v3")
SA = ee.FeatureCollection("users/liu02034/SanAntonioTX_Project")
NLCD = ee.ImageCollection("USGS/NLCD_RELEASES/2019_REL/NLCD")

region = SA

Map.addLayer(SA, {}, 'SA', False)

GDriveOutputImgFolder = 'GEEOutputs'

#tree canopy
FirstYear = "2015"
LastYear = "2015"
SelectedVariableName = 'tree_canopy_cover'
coll1 =  Tree_cover.filterDate(FirstYear+'-01-01', LastYear+'-12-31').filterBounds(region).select([SelectedVariableName])

print("ori",coll1)
#Map.addLayer(coll1, {}, 'tree_cover', False)

mosaic = coll1.mosaic()
Map.addLayer(mosaic, {}, 'mosaic', False)
print("mosaic",mosaic)

#NLCD 2016
FirstYear = "2016"
LastYear = "2016"
SelectedVariableName = 'landcover'
coll1_NLCD =  NLCD.filterDate(FirstYear+'-01-01', LastYear+'-12-31').filterBounds(region).select([SelectedVariableName])
print("ori_NLCD",coll1_NLCD)

mosaic_NLCD = coll1_NLCD.mosaic()
Map.addLayer(mosaic_NLCD, {}, 'mosaic_NLCD', False)
print("mosaic_NLCD",mosaic_NLCD)

scale = 30;
maxPixels = 1.0e13

task = ee.batch.Export.image.toDrive(
        mosaic,
        description='Tree_canopy_NASA_30m_SA',
        folder=GDriveOutputImgFolder,
        region=region.geometry(),
        scale=scale,
        maxPixels=maxPixels,
    )

task.start()

task = ee.batch.Export.image.toDrive(
        mosaic_NLCD,
        description='NLCD_2016_30m_SA',
        folder=GDriveOutputImgFolder,
        region=region.geometry(),
        scale=scale,
        maxPixels=maxPixels,
    )

task.start()

Map